In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import utils
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
data = np.loadtxt('../input/data-first/train-1.txt', dtype=str, delimiter = '\n')
#Valid = np.loadtxt('../input/train2/train-2.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


In [3]:
blacks = 0
whites = 0

In [4]:
for elem in data:
    if elem[0] == 'b':
        blacks += 1
    elif elem[0] == 'w':
        whites += 1

In [5]:
curBlack = 0
curWhite = 0

In [6]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [7]:
def ToPos(turn, bias_x, bias_y):
    letter = ord(turn[0]) - ord('a')
    if 0 <= letter + bias_x <= 14:
        letter += bias_x
    else:
        return -1, -1
    
    num = int(turn[1:]) - 1
    if 0 <= num + bias_y <= 14:
        num += bias_y
    else:
        return -1, -1

    return letter, num

def Transform(game, color, turn, bias):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard

    res = []
    labels = []

    for direction in range(8):
        if direction == 0:
            bias_x, bias_y = bias, 0
        if direction == 1:
            bias_x, bias_y = -bias, 0
        if direction == 2:
            bias_x, bias_y = 0, -bias
        if direction == 3:
            bias_x, bias_y = 0, bias
        if direction == 4:
            bias_x, bias_y = bias, bias
        if direction == 5:
            bias_x, bias_y = bias, -bias
        if direction == 6:
            bias_x, bias_y = -bias, -bias
        if direction == 7:
            bias_x, bias_y = -bias, bias

        curPlayer = 1
        whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
        blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
        for i in range(1, turn + 1):
            if len(game[i]) < 2:
                continue
            CurTurn = ToPos(game[i], bias_x, bias_y)
            if CurTurn[0] == -1 and CurTurn[1] == -1:
                continue
            if curPlayer == 1:
                blackBoard[CurTurn] = 1
            else:
                whiteBoard[CurTurn] = -1
            curPlayer *= -1

        curBoard = np.zeros(shape=(3, 15, 15))
        curBoard[0,:] = playerBoard
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        label = ToPos(game[turn + 1], bias_x, bias_y)
        if label[0] == -1 and label[1] == -1:
            continue

        res.append(curBoard)
        labels.append(label[0] * 15 + label[1])

    return np.asarray(res), np.asarray(labels)

In [8]:
NumOfFine = 0
for elem in data:
    if elem[0] != 'd' and elem[0] != 'u':
        NumOfFine += 1

In [9]:
Dataset = np.ndarray(shape=NumOfFine, dtype='U765')
idx = 0
for elem in data:
    if elem[0] != 'd' and elem[0] != 'u':
        Dataset[idx] = elem
        idx += 1
print(Dataset.shape)

(1941123,)


In [10]:
def GetRandom(data, bias):
    game = []
    while len(game) <= 4:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = Transform(game, color, turn * 2, bias)
    else:
        board, label = Transform(game, color, turn * 2 + 1, bias)

    return board, label

def Generator(size, data):
    Batch = np.ndarray(shape=(0, 3, 15, 15), dtype=np.int8)
    BatchLabels = np.ndarray(shape=0, dtype=np.long)
    for i in range(size):
        bias = np.random.randint(1, 4)
        res, labels = GetRandom(data, bias)
        Batch = np.concatenate((Batch, res), axis=0)
        BatchLabels = np.concatenate((BatchLabels, labels), axis=0)

    return torch.from_numpy(Batch).float(), torch.from_numpy(BatchLabels)

In [11]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=2),
            nn.BatchNorm2d(128, affine=False),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, stride=1, padding=0),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=4, stride=1, padding=0),
            nn.ReLU())
        self.fc = nn.Linear(16 * 9 * 9, 15 * 15)
        self.drop = nn.Dropout(0.25)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(-1, 16 * 9 * 9)
        x = self.drop(x)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)

        return x

In [12]:
DatasetTrain = Dataset[0 : (NumOfFine * 8) // 10]
DatasetValid = Dataset[(NumOfFine * 8) // 10 : NumOfFine]

net = ConvNet()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
CntBatch = 400000
CntEpoch = 2
BatchSize = 16
optimizer = optim.Adam(net.parameters(), lr=0.0001)
net.load_state_dict(torch.load("../input/model6/model5-4.dms", map_location=device))

In [13]:
for epoch in range(CntEpoch):
    running_loss = 0.0
    
    for i in range(CntBatch):
        inputs, labels = Generator(BatchSize, DatasetTrain)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 10000 == 9999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10000))
            running_loss = 0.0
print('Finished Training')

[1, 10000] loss: 1.903
[1, 20000] loss: 1.900
[1, 30000] loss: 1.888
[1, 40000] loss: 1.899
[1, 50000] loss: 1.899
[1, 60000] loss: 1.896
[1, 70000] loss: 1.891
[1, 80000] loss: 1.894
[1, 90000] loss: 1.891
[1, 100000] loss: 1.901
[1, 110000] loss: 1.895
[1, 120000] loss: 1.889
[1, 130000] loss: 1.894
[1, 140000] loss: 1.898
[1, 150000] loss: 1.891
[1, 160000] loss: 1.883
[1, 170000] loss: 1.888
[1, 180000] loss: 1.894
[1, 190000] loss: 1.888
[1, 200000] loss: 1.888
[1, 210000] loss: 1.889
[1, 220000] loss: 1.879
[1, 230000] loss: 1.882
[1, 240000] loss: 1.883
[1, 250000] loss: 1.885
[1, 260000] loss: 1.882
[1, 270000] loss: 1.885
[1, 280000] loss: 1.892
[1, 290000] loss: 1.889
[1, 300000] loss: 1.876
[1, 310000] loss: 1.873
[1, 320000] loss: 1.881
[1, 330000] loss: 1.885
[1, 340000] loss: 1.887
[1, 350000] loss: 1.874
[1, 360000] loss: 1.873
[1, 370000] loss: 1.877
[1, 380000] loss: 1.876
[1, 390000] loss: 1.873
[1, 400000] loss: 1.877
[2, 10000] loss: 1.869
[2, 20000] loss: 1.873
[2,

In [14]:
torch.save(net.state_dict(), "./model5")

In [15]:
correct = 0
total = 0
for i in range(50000):
    if i % 10000 == 9999:
        print('Accuracy of the network on the 200 positions: %5f %%' % (100 * correct / total))    
    inputs, labels = Generator(BatchSize, DatasetValid)
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 50000 positions: %d %%' % (100 * correct / total))

Accuracy of the network on the 200 positions: 46.083357 %
Accuracy of the network on the 200 positions: 45.872773 %
Accuracy of the network on the 200 positions: 45.887109 %
Accuracy of the network on the 200 positions: 45.888661 %
Accuracy of the network on the 200 positions: 45.875111 %
Accuracy of the network on the 50000 positions: 45 %
